# Sigma example notebook

This notebook should showcase some use cases with regards to Sigma.

In [ ]:
!pip install -q timesketch_api_client

In [ ]:
from timesketch_api_client import search
from picatrix.lib import state as state_lib

import requests
import io
import altair as alt
import numpy as np
import pandas as pd
from timesketch_api_client import config
from timesketch_import_client import helper
from timesketch_import_client import importer
from timesketch_api_client import search

# Create the sketch and import some data

First we want to create a sketch and import a small data set. It is recommended to do that with the dev docker container.

In [ ]:
ts_client = config.get_client()

We create a new sketch

That will be called test and will be filled with some test data from the Github repository.

The downloaded csv will be read into a pandas dataframe for further processing.

In [ ]:
sketch = ts_client.create_sketch(name="test")

In [ ]:
url = "https://raw.githubusercontent.com/google/timesketch/master/test_tools/test_events/sigma_events.csv" 
download = requests.get(url).content

In [ ]:
df = pd.read_csv(io.StringIO(download.decode('utf-8')))

In [ ]:
df.head(4)

## Import the data to timesketch

The easiest way to do that is using the import client as shown below.

In [ ]:
import_helper = helper.ImportHelper() 

with importer.ImportStreamer() as streamer:
  streamer.set_sketch(sketch)
  streamer.set_config_helper(import_helper) 

  streamer.set_timeline_name('sigma_events.csv')
  streamer.add_data_frame(df)


# Fetch Rules

Now we want to fetch all the rules installed on the Timesketch instance. For the dev docker container we only expect one rule.


In [ ]:
rules = ts_client.list_sigma_rules()

In [ ]:
for rule in rules:
    print(f'ID: {rule.id} Title: {rule.title}')

In [ ]:
rule1 = rules[0]

The API also gives us the elastic search query we could use to explore the data.

In [ ]:
rule1.es_query

# Single rule

Next we want to fetch a single rule by a given uuid.

In [ ]:
single_rule = ts_client.get_sigma_rule(rule_uuid='5266a592-b793-11ea-b3de-0242ac130004')

In [ ]:
single_rule.id

In [ ]:
single_rule.es_query

# Text Sigma examples

The API also provides the option to provide a Sigma rule by text and the backend will parse it with the installed Sigma mappings. This can be especially helpful when developing new rules before installing and exposing them to all the other user of a Timesketch instance.

In [ ]:
MOCK_SIGMA_RULE = r"""
title: Suspicious Installation of Zenmap
id: 5266a592-b793-11ea-b3de-0242ac130004
description: Detects suspicious installation of Zenmap
references:
    - https://rmusser.net/docs/ATT&CK-Stuff/ATT&CK/Discovery.html
author: Alexander Jaeger
date: 2020/06/26
modified: 2021/01/01
logsource:
    product: linux
    service: shell
detection:
    keywords:
        # Generic suspicious commands
        - '*apt-get install zmap*'
    condition: keywords
falsepositives:
    - Unknown
level: high
"""

In [ ]:
rule_by_text = ts_client.get_sigma_rule_by_text(MOCK_SIGMA_RULE)

In [ ]:
rule_by_text.es_query

In [ ]:
rule_by_text.references

# Analyzer

Timesketch also has a Sigma analyzer that will add labels to all matching events. To do so you need a Timeline object to run the analyzer. The Analyzer then will take all rules installed on the Timesketch instance and go over the Timeline.

In [ ]:
timelines = sketch.list_timelines()
timeline = None
for timeline_ in timelines:
  if timeline_.name == 'sigma_events.csv':
    timeline = timeline_
    break

In [ ]:
result = timeline.run_analyzer(analyzer_name='sigma', ignore_previous=True)


In [ ]:
sketch.get_analyzer_status()

This should show something like:
```
[{'index': <timesketch_api_client.index.SearchIndex at 0x126a09820>,
  'timeline_id': 1,
  'session_id': 1,
  'analyzer': 'sigma',
  'results': 'Applied 1 tags\n* lnx_susp_zenmap.yml: 1\nProblematic rules:',
  'status': 'DONE'}]
```